# Covid-19 Open API 이용 -> Data 획득 -> DB 저장

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
key_fd = open('gov_data_api_key.txt', mode='r')
govapi_key = key_fd.read(100)
key_fd.close()

## 1. 시도발생 현황 region 테이블에 저장하기

In [97]:
start_date = '20200302'
end_date = '20200302'
page = 1

In [98]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url = f'{corona_url}?ServiceKey={govapi_key}&pageNo={page}&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [99]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
resultCode = soup.find('resultCode').get_text()
if resultCode == '00':
    print(soup.find('totalCount').string)
elif resultCode == '99':
    print(soup.find('resultMsg').string)

19


In [100]:
createDt_list,deathCnt_list,defCnt_list,gubun_list,incDec_list = [],[],[],[],[]
isolClearCnt_list,isolIngCnt_list,localOccCnt_list,overFlowCnt_list = [],[],[],[]
qurRate_list,seq_list,stdDay_list,updateDt_list = [],[],[],[]
items = soup.find_all('item')
for item in items:
    createDt_list.append(item.find('createDt').string if item.find('createDt') else '')
    deathCnt_list.append(item.find('deathCnt').string if item.find('deathCnt') else '')
    defCnt_list.append(item.find('defCnt').string if item.find('defCnt') else '')
    gubun_list.append(item.find('gubun').string if item.find('gubun') else '')
    incDec_list.append(item.find('incDec').string if item.find('incDec') else '')
    isolClearCnt_list.append(item.find('isolClearCnt').string if item.find('isolClearCnt') else '')
    isolIngCnt_list.append(item.find('isolIngCnt').string if item.find('isolIngCnt') else '')
    localOccCnt_list.append(item.find('localOccCnt').string if item.find('localOccCnt') else '')
    overFlowCnt_list.append(item.find('overFlowCnt').string if item.find('overFlowCnt') else '')
    qurRate_list.append(item.find('qurRate').string if item.find('qurRate') else '')
    seq_list.append(item.find('seq_list').string if item.find('seq_list') else '')
    stdDay_list.append(item.find('stdDay').string if item.find('stdDay') else '')
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else '')

df = pd.DataFrame({
    '등록시간':createDt_list, '사망자':deathCnt_list, '확진자':defCnt_list,
    '광역시도':gubun_list, '전일대비':incDec_list, '격리해제':isolClearCnt_list, 
    '격리중':isolIngCnt_list, '지역발생':localOccCnt_list,'해외유입':overFlowCnt_list,
    '10만명당':qurRate_list, 'ID':seq_list, '기준시간':stdDay_list, '수정시간':updateDt_list
})
df

,등록시간,사망자,확진자,광역시도,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,ID,기준시간,수정시간
0,2020-03-02 19:27:57.57,0,,검역,0,,,,,None,,2020년 3월 1일 16시,2020-03-03 20:04:50.50
1,2020-03-02 19:27:57.57,0,,제주,0,,,,,None,,2020년 3월 1일 16시,None
2,2020-03-02 19:27:57.57,0,,경남,4,,,,,None,,2020년 3월 1일 16시,None
3,2020-03-02 19:27:57.57,8,,경북,67,,,,,None,,2020년 3월 1일 16시,None
4,2020-03-02 19:27:57.57,0,,전남,0,,,,,None,,2020년 3월 1일 16시,None
5,2020-03-02 19:27:57.57,0,,전북,1,,,,,None,,2020년 3월 1일 16시,None
6,2020-03-02 19:27:57.57,0,,충남,13,,,,,None,,2020년 3월 1일 16시,None
7,2020-03-02 19:27:57.57,0,,충북,1,,,,,None,,2020년 3월 1일 16시,None
8,2020-03-02 19:27:57.57,0,,강원,8,,,,,None,,2020년 3월 1일 16시,None
9,2020-03-02 19:27:57.57,1,,경기,7,,,,,None,,2020년 3월 1일 16시,None


### 테이블 생성

In [72]:
import sqlite3
conn = sqlite3.connect('../db/covid.db')
cur = conn.cursor()

In [74]:
sql_table = '''create table if not exists region (
  rid integer not null primary key autoincrement,
  stdDay text not null,
  deathCnt int not null,
  defCnt int not null,
  gubun text not null,
  incDec int not null,
  isolClearCnt int not null,
  isolIngCnt int not null,
  localOccCnt int not null,
  overFlowCnt int not null,
  qurRate float);
'''
cur.execute(sql_table)
conn.commit()

### 하나의 레코드를 DB에 삽입

In [71]:
# 기준시간을 YYYY-MM-DD로 변환
def change_date(x):
    y = x.split(' ')
    month = y[1][:-1] if len(y[1][:-1]) == 2 else '0'+y[1][:-1]
    day = y[2][:-1] if len(y[2][:-1]) == 2 else '0'+y[2][:-1]
    return f'{y[0][:-1]}-{month}-{day}'

In [43]:
params = [change_date(df['기준시간'][0]), int(df['사망자'][0]), int(df['확진자'][0]), 
          df['광역시도'][0], int(df['전일대비'][0]), int(df['격리해제'][0]), 
          int(df['격리중'][0]), int(df['지역발생'][0]), int(df['해외유입'][0])]
params.append(None if df['10만명당'][0]=='-' else float(df['10만명당'][0]))
params

['2020-10-31', 0, 1766, '검역', 25, 1467, 299, 0, 25, None]

In [19]:
sql_insert = '''insert into region(stdDay, deathCnt, defCnt, gubun, incDec, isolClearCnt,
isolIngCnt, localOccCnt, overFlowCnt, qurRate) values(?,?,?,?,?,?,?,?,?,?);
'''
cur.execute(sql_insert, params)
conn.commit()

### 하루 데이터 19건을 DB에 삽입

In [44]:
for i in range(19):
    params = [change_date(df['기준시간'][i]), int(df['사망자'][i]), int(df['확진자'][i]), 
              df['광역시도'][i], int(df['전일대비'][i]), int(df['격리해제'][i]), 
              int(df['격리중'][i]), int(df['지역발생'][i]), int(df['해외유입'][i])]
    params.append(None if df['10만명당'][i]=='-' else float(df['10만명당'][i]))
    cur.execute(sql_insert, params)
    conn.commit()

### 최종적으로 3월1일부터 12월21까지 DB에 write하는 코드

In [35]:
from tqdm import tqdm_notebook

In [73]:
sql_insert = '''insert into region(stdDay, deathCnt, defCnt, gubun, incDec, isolClearCnt,
isolIngCnt, localOccCnt, overFlowCnt, qurRate) values(?,?,?,?,?,?,?,?,?,?);
'''
month_dict = {3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:21}

In [80]:
for k, v in tqdm_notebook(month_dict.items()):
    month = str(k) if k >= 10 else f'0{k}'
    for i in range(1, v+1):
        day = str(i) if i >= 10 else f'0{i}'
        date = f'2020{month}{day}'
        #print(date)
        page = 1
        url = f'{corona_url}?ServiceKey={govapi_key}&pageNo={page}&numOfRows=10&startCreateDt={date}&endCreateDt={date}'
        soup = BeautifulSoup(requests.get(url).text, 'xml')

        items = soup.find_all('item')
        item_count = len(items)
        if item_count == 0:
            continue
        for index, item in enumerate(items):
            if item_count > 30 and index >= int(item_count/2):
                break
            stdDay = change_date(item.find('stdDay').string)
            deathCnt = int(item.find('deathCnt').string) if item.find('deathCnt') else 0
            defCnt = int(item.find('defCnt').string) if item.find('defCnt') else 0
            gubun = item.find('gubun').string
            incDec = int(item.find('incDec').string)
            isolClearCnt = int(item.find('isolClearCnt').string) if item.find('isolClearCnt') else 0
            isolIngCnt = int(item.find('isolIngCnt').string) if item.find('isolIngCnt') else 0
            localOccCnt = int(item.find('localOccCnt').string) if item.find('localOccCnt') else 0
            overFlowCnt = int(item.find('overFlowCnt').string) if item.find('overFlowCnt') else 0
            qurRate = None
            if item.find('qurRate'):
                qur = item.find('qurRate').string
                if qur != None and qur.count('.') == 2:
                    qur = qur[:-1]
                #print(qur)
                if qur != None and qur[0] in '0123456789':
                    qurRate = float(qur)
        
            params = [stdDay, deathCnt, defCnt, gubun, incDec, isolClearCnt, isolIngCnt, 
                    localOccCnt, overFlowCnt, qurRate]
            #print(params)
            cur.execute(sql_insert, params)
            conn.commit()

In [47]:
cur.close()
conn.close()

## 2. 연령별.성별 데이터 

In [61]:
start_date = '20200301'
end_date = '20200301'
page = 1

In [62]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson'
url = f'{corona_url}?ServiceKey={govapi_key}&pageNo={page}&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [63]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
resultCode = soup.find('resultCode').get_text()
if resultCode == '00':
    print(soup.find('totalCount').string)
elif resultCode == '99':
    print(soup.find('resultMsg').string)

0


In [64]:
soup

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>

In [53]:
createDt_list,confCase_list,confCaseRate_list,death_list,deathRate_list = [],[],[],[],[]
gubun_list,criticalRate_list,seq_list,updateDt_list = [],[],[],[]
items = soup.find_all('item')
for item in items:
    createDt_list.append(item.find('createDt').string if item.find('createDt') else '')
    confCase_list.append(item.find('confCase').string if item.find('confCase') else '')
    confCaseRate_list.append(item.find('confCaseRate').string if item.find('confCaseRate') else '')
    death_list.append(item.find('death').string if item.find('death') else '')
    deathRate_list.append(item.find('deathRate').string if item.find('deathRate') else '')
    criticalRate_list.append(item.find('criticalRate').string if item.find('criticalRate') else '')
    gubun_list.append(item.find('gubun').string if item.find('gubun') else '')
    seq_list.append(item.find('seq').string if item.find('seq') else '')
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else '')

df = pd.DataFrame({
    '등록시간':createDt_list, '확진자':confCase_list, '확진률':confCaseRate_list,
    '사망자':death_list, '사망률':deathRate_list, '치명률':criticalRate_list, 
    '구분':gubun_list, 'ID':seq_list, '수정시간':updateDt_list
})
df

,등록시간,확진자,확진률,사망자,사망률,치명률,구분,ID,수정시간
0,2020-12-21 14:18:51.88,1684,3.33,0,0.00,0,0-9,5565,null
1,2020-12-21 14:18:51.88,3150,6.23,0,0.00,0,10-19,5564,null
2,2020-12-21 14:18:51.88,8552,16.9,0,0.00,0,20-29,5563,null
3,2020-12-21 14:18:51.88,6414,12.68,3,0.43,0.05,30-39,5562,null
4,2020-12-21 14:18:51.88,7099,14.03,6,0.86,0.08,40-49,5561,null
5,2020-12-21 14:18:51.88,9402,18.58,29,4.15,0.31,50-59,5560,null
6,2020-12-21 14:18:51.88,7883,15.58,84,12.03,1.07,60-69,5559,null
7,2020-12-21 14:18:51.879,3979,7.87,204,29.23,5.13,70-79,5558,null
8,2020-12-21 14:18:51.879,2428,4.8,372,53.30,15.32,80 이상,5557,null
9,2020-12-21 14:18:51.879,26262,51.91,340,48.71,1.29,여성,5556,null


In [54]:
conn = sqlite3.connect('../db/covid.db')
cur = conn.cursor()

In [55]:
sql_table = '''create table if not exists agender (
  agid integer not null primary key autoincrement,
  stdDay text not null,
  confCase int not null,
  confCaseRate float not null,
  death int not null,
  deathRate float not null,
  criticalRate float not null,
  gubun text not null,
  seq int,
  updateDt text);
'''
cur.execute(sql_table)
conn.commit()

In [59]:
month_dict = {3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:21}
for k, v in tqdm_notebook(month_dict.items()):
    month = str(k) if k >= 10 else f'0{k}'
    print(f'{k}월: ', end='')
    for i in range(1, v+1):
        day = str(i) if i >= 10 else f'0{i}'
        print(f'{i}-', end='')
        date = f'2020{month}{day}'
        page = 1
        url = f'{corona_url}?ServiceKey={govapi_key}&pageNo={page}&numOfRows=10&startCreateDt={date}&endCreateDt={date}'
        soup = BeautifulSoup(requests.get(url).text, 'xml')

        items = soup.find_all('item')
        item_count = len(items)
        print(f'{item_count}, ', end='')
    print()

3월: 1-0, 2-0, 3-0, 4-0, 5-0, 6-0, 7-0, 8-0, 9-0, 10-0, 11-0, 12-0, 13-0, 14-0, 15-0, 16-0, 17-0, 18-0, 19-0, 20-0, 21-0, 22-0, 23-0, 24-0, 25-0, 26-0, 27-0, 28-0, 29-0, 30-0, 31-0, 
4월: 1-0, 2-11, 3-0, 4-0, 5-0, 6-0, 7-12, 8-11, 9-11, 10-11, 11-11, 12-11, 13-11, 14-11, 15-11, 16-11, 17-11, 18-11, 19-11, 20-11, 21-11, 22-11, 23-11, 24-11, 25-11, 26-11, 27-11, 28-11, 29-11, 30-11, 
5월: 1-11, 2-11, 3-11, 4-11, 5-11, 6-11, 7-11, 8-11, 9-11, 10-11, 11-11, 12-11, 13-11, 14-11, 15-11, 16-11, 17-11, 18-11, 19-11, 20-11, 21-11, 22-0, 23-11, 24-11, 25-11, 26-11, 27-11, 28-11, 29-11, 30-11, 31-11, 
6월: 1-0, 2-11, 3-11, 4-11, 5-11, 6-11, 7-11, 8-11, 9-11, 10-11, 11-11, 12-11, 13-11, 14-11, 15-0, 16-11, 17-11, 18-11, 19-11, 20-11, 21-11, 22-11, 23-11, 24-11, 25-11, 26-11, 27-11, 28-11, 29-11, 30-11, 
7월: 1-11, 2-11, 3-11, 4-11, 5-11, 6-11, 7-11, 8-11, 9-11, 10-11, 11-11, 12-11, 13-11, 14-11, 15-11, 16-11, 17-11, 18-11, 19-11, 20-11, 21-11, 22-11, 23-11, 24-11, 25-11, 26-11, 27-11, 28-11, 29-11, 30-

In [68]:
sql_insert = '''insert into agender(stdDay, confCase, confCaseRate, death, deathRate,
criticalRate, gubun, seq, updateDt) values(?,?,?,?,?,?,?,?,?);
'''

In [69]:
month_dict = {4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:21}
for k, v in tqdm_notebook(month_dict.items()):
    month = str(k) if k >= 10 else f'0{k}'
    for i in range(1, v+1):
        day = str(i) if i >= 10 else f'0{i}'
        date = f'2020{month}{day}'
        page = 1
        url = f'{corona_url}?ServiceKey={govapi_key}&pageNo={page}&numOfRows=10&startCreateDt={date}&endCreateDt={date}'
        soup = BeautifulSoup(requests.get(url).text, 'xml')

        items = soup.find_all('item')
        item_count = len(items)
        if item_count == 0:
            continue

        for item in items:
            createDt = item.find('createDt').string.split(' ')[0]
            confCase = int(item.find('confCase').string)
            confCaseRate = float(item.find('confCaseRate').string)
            death = int(item.find('death').string)
            deathRate = float(item.find('deathRate').string)
            criticalRate = float(item.find('criticalRate').string)
            gubun = item.find('gubun').string
            seq = int(item.find('seq').string)
            updateDt = item.find('updateDt').string

            params = [createDt, confCase, confCaseRate, death, deathRate, criticalRate,
                      gubun,seq,updateDt]
            cur.execute(sql_insert, params)
            conn.commit()

In [70]:
cur.close()
conn.close()